In [9]:
import numpy as np
import scipy.linalg
from collections import defaultdict, namedtuple, Counter
from itertools import combinations
import numpy as np
import scipy.stats as stats
import sys
from os import listdir
import json
import random
import csv
import sys

In [10]:
dataset = 'ssc.hg38'
subtype = 'current' #current/life
num_trials = 1000
interval_chrom, interval_start_pos, interval_end_pos = None, None, None

output_file = dataset

In [12]:
is_reverse_scored = [False]*8 + [True] + [False]*9 + [True]*22

if interval_chrom is not None:
	output_file+= '.chr%s' % interval_chrom
if interval_start_pos is not None or interval_end_pos is not None:
	output_file += '.%d-%d' % (interval_start_pos, interval_end_pos)

def header_to_inds(header):
	header = header.strip().split('\t')
	return [header[i][:-4] for i in range(5, len(header)-3, 2)]

In [13]:
with open('../../PhasingFamilies/%s/sibpairs.json' % dataset, 'r') as f:
	sibpairs = json.load(f)

print('Overall')
print('families', len(set([x['family'].split('.')[0] for x in sibpairs])))
print('sibpairs', len(sibpairs))
num_sibpairs = len(sibpairs)

Overall
families 1905
sibpairs 1905


In [67]:
def apply_interval_filter(chrom, start_pos, end_pos):
	if interval_start_pos is not None or interval_end_pos is not None:
		start_pos = np.clip(start_pos, interval_start_pos, interval_end_pos)
		end_pos = np.clip(end_pos, interval_start_pos, interval_end_pos)
	is_ok = (interval_chrom is None or interval_chrom == chrom) and (end_pos-start_pos>0)
	return is_ok, start_pos, end_pos

def process_phase_file(sibpair):
	with open('../../PhasingFamilies/%s/%s.phased.txt' % (sibpair['phase_dir'], sibpair['family']), 'r')  as f:
		header = next(f) # skip header
		inds = header_to_inds(header)
		sib1_ind_index, sib2_ind_index = inds.index(sibpair['sibling1']), inds.index(sibpair['sibling2'])
		sib1_mat_index, sib2_mat_index = 4+(2*sib1_ind_index), 4+(2*sib2_ind_index)
		sib1_pat_index, sib2_pat_index = 5+(2*sib1_ind_index), 5+(2*sib2_ind_index)
		sib_phase_indices = [sib1_mat_index, sib2_mat_index, sib1_pat_index, sib2_pat_index]

		current_chrom, current_start_pos, current_end_pos, current_state = None, None, None, None
		for line in f:
			pieces = line.strip().split('\t')
			chrom = pieces[0][3:]
			start_pos, end_pos = [int(x) for x in pieces[-2:]]
			state = np.array([int(x) for x in pieces[1:-2]])[sib_phase_indices]

			if current_chrom is None:
				current_chrom, current_start_pos, current_end_pos, current_state = chrom, start_pos, end_pos, state
			elif current_chrom != chrom or np.any(current_state != state):
				is_ok, current_start_pos, current_end_pos = apply_interval_filter(current_chrom, current_start_pos, current_end_pos)
				if is_ok:
					yield current_chrom, current_start_pos, current_end_pos, current_state
				current_chrom, current_start_pos, current_end_pos, current_state = chrom, start_pos, end_pos, state
			else:
				current_end_pos = end_pos
		is_ok, current_start_pos, current_end_pos = apply_interval_filter(current_chrom, current_start_pos, current_end_pos)
		if is_ok:
			yield current_chrom, current_start_pos, current_end_pos, current_state

In [68]:
# pull intervals
positions = set()
for sibpair in sibpairs:
	#print(sibpair)
	for chrom, start_pos, end_pos, state in process_phase_file(sibpair):
		#print(chrom, start_pos, end_pos, state)
		positions.add((chrom, start_pos))
		positions.add((chrom, end_pos))

positions = sorted(positions, key=lambda x: (int(x[0]), x[1]) if x[0].isdigit() else x)
chroms, interval_starts, interval_ends = [], [], []
prev_chrom, prev_pos = None, None
for c, p in positions:
	if prev_chrom is not None and prev_chrom == c:
		chroms.append(c)
		interval_starts.append(prev_pos)
		interval_ends.append(p)
	prev_chrom, prev_pos = c, p


interval_starts = np.array(interval_starts)
interval_ends = np.array(interval_ends)
num_intervals = len(interval_starts)
print('intervals', num_intervals)

intervals 568577


In [69]:
# sibpair, interval
is_mat_match = np.zeros((num_sibpairs, num_intervals), dtype=int)
is_pat_match = np.zeros((num_sibpairs, num_intervals), dtype=int)

interval_start_to_index = dict([((chrom, x), i) for i, (chrom, x) in enumerate(zip(chroms, interval_starts))])
interval_end_to_index = dict([((chrom, x), i) for i, (chrom, x) in enumerate(zip(chroms, interval_ends))])

for sibpair_index, sibpair in enumerate(sibpairs):
	for chrom, start_pos, end_pos, state in process_phase_file(sibpair):
		start_index, end_index = interval_start_to_index[(chrom, start_pos)], interval_end_to_index[(chrom, end_pos)]+1

		if state[0]==-1 or state[1]==-1:
			pass
		elif state[0]==state[1]:
			is_mat_match[sibpair_index, start_index:end_index] = 1
		else:
			is_mat_match[sibpair_index, start_index:end_index] = -1
			
		if state[2]==-1 or state[3]==-1:
				pass
		elif state[2]==state[3]:
			is_pat_match[sibpair_index, start_index:end_index] = 1
		else:
			is_pat_match[sibpair_index, start_index:end_index] = -1


is_ok = interval_ends - interval_starts > 1
interval_starts = interval_starts[is_ok]
interval_ends = interval_ends[is_ok]
chroms = np.array([int(c) for c in chroms])[is_ok]
is_mat_match = is_mat_match[:, is_ok]
is_pat_match = is_pat_match[:, is_ok]
num_intervals = np.sum(is_ok)

print(is_mat_match.shape, is_pat_match.shape)

np.save('../permutation_tests/phen.%s.chroms.npy' % output_file, chroms)
np.save('../permutation_tests/phen.%s.intervals.npy' % output_file, np.array([interval_starts, interval_ends]))


(6925, 386437) (6925, 386437)


In [81]:
# take into account sibling structure across quads
individuals = sorted(set([x['sibling1'] for x in sibpairs] + [x['sibling2'] for x in sibpairs]))
ind_to_index = dict([(x, i) for i, x in enumerate(individuals)])
sibling1_indices = np.array([ind_to_index[x['sibling1']] for x in sibpairs])
sibling2_indices = np.array([ind_to_index[x['sibling2']] for x in sibpairs])

A = np.random.randint(0, high=2, size=(num_trials+1, len(individuals), 2))
X1 = (A[:, sibling1_indices, 0] == A[:, sibling2_indices, 0]).astype(int)
X2 = (A[:, sibling1_indices, 1] == A[:, sibling2_indices, 1]).astype(int)

# randomly flip IBD in sibpairs
#X1 = np.random.randint(0, high=2, size=(num_trials+1, len(sibpairs)))
#X2 = np.random.randint(0, high=2, size=(num_trials+1, len(sibpairs)))

X1[X1==0] = -1
X2[X2==0] = -1

# first entry is actual IBD relationships
X1[0, :] = 1
X2[0, :] = 1

print('ready')

ready


In [21]:
phen_index = 0


In [22]:
sample_to_affected = dict()

if dataset == 'ssc.hg38':
	output_file = output_file + '.' + subtype

	old_id_to_new_id = dict()
	# p1 vs s1 are random, but it's ok since we know they're all quads
	for sibpair in sibpairs:
		old_id_to_new_id['%s.p1' % sibpair['family']] = sibpair['sibling1']
		old_id_to_new_id['%s.s1' % sibpair['family']] = sibpair['sibling2']

	with open('../../PhasingFamilies/phenotypes/ssc/proband.data/scq_%s_raw.csv' % subtype, 'r') as f:
		reader = csv.reader(f)
		for pieces in reader:
			phen = pieces[2+phen_index]
			if (pieces[0] in old_id_to_new_id) and (phen=='yes' or phen=='no'):
				sample_to_affected[old_id_to_new_id[pieces[0]]] = 1 if phen =='yes' else 0

	with open('../../PhasingFamilies/phenotypes/ssc/designated.unaffected.sibling.data/scq_%s_raw.csv' % subtype, 'r') as f:
		reader = csv.reader(f)
		for pieces in reader:
			phen = pieces[2+phen_index]
			if (pieces[0] in old_id_to_new_id) and (phen=='yes' or phen=='no'):
				sample_to_affected[old_id_to_new_id[pieces[0]]] = 1 if phen =='yes' else 0

In [23]:
if dataset == 'spark':
    sample_to_affected = dict()
    with open('../../PhasingFamilies/phenotypes/spark_v5/spark_v5-scq-prep.csv', 'r') as f:
        reader = csv.reader(f)
        for pieces in reader:
            phen = pieces[13+phen_index]
            if phen=='1.0' or phen=='0.0':
                sample_to_affected[pieces[2]] = '1' if phen =='1.0' else '0'

In [24]:
num_affected = np.array([-1 if (x['sibling1'] not in sample_to_affected or x['sibling2'] not in sample_to_affected) else int(sample_to_affected[x['sibling1']])+int(sample_to_affected[x['sibling2']]) for x in sibpairs])
#num_affected = np.array([-1 if (x.family + '.p1' not in sample_to_affected or x.family + '.s1' not in sample_to_affected) else int(sample_to_affected[x.family + '.p1'])+int(sample_to_affected[x.family + '.s1']) for x in sibpairs])
print(Counter(num_affected))
na = 2 if is_reverse_scored[phen_index] else 0
print(na, np.sum(num_affected==na), end=' ')


Counter({-1: 1265, 2: 572, 1: 68})
0 0 

In [145]:

is_match_reduced, reduced_inverse = np.unique(np.vstack((is_mat_match[num_affected==na, :],
                                                                 is_pat_match[num_affected==na, :])), axis=1, return_inverse=True)
print(is_mat_match.shape[1], is_match_reduced.shape)


386437 (2118, 160661)


In [ ]:
num_intervals = is_match_reduced.shape[1]

all_pvalues_reduced = np.zeros((num_intervals, ))

# trial, interval, mat/pat
rand_pvalue = np.hstack((X1[:, num_affected==na], X2[:, num_affected==na])).dot(is_match_reduced)

# -------------------- implementing Westfall-Young max T stepdown procedure

# indices are sorted along interval axis from interval with most IBD sharing
# to least IBD sharing

orig_indices = np.flip(np.argsort(rand_pvalue[0, :]))

max_t_k = np.zeros((num_trials+1, num_intervals+1))
max_t_k[:, -1] = np.min(rand_pvalue, axis=1)
for i, j in list(reversed(list(enumerate(orig_indices)))):
	max_t_k[:, i] = np.maximum(max_t_k[:, i+1], rand_pvalue[:, j])
max_t_k = max_t_k[:, :-1]
				
assert np.all(max_t_k[0, :] == rand_pvalue[0, orig_indices])

# calculate pi(j)
pvalues = np.sum(max_t_k[1:, :] >= np.tile(max_t_k[0, :], (num_trials, 1)), axis=0)/num_trials
pvalues = np.array([np.max(pvalues[:(i+1)]) for i in np.arange(num_intervals)])
all_pvalues_reduced[orig_indices] = pvalues

all_pvalues = all_pvalues_reduced[reduced_inverse]
print(all_pvalues.shape)

In [ ]:
print(np.min(all_pvalues))

In [ ]:
np.save('../permutation_tests/scq%d.%s.npy' % (phen_index+1, output_file), all_pvalues)
